Download GNPS data <br>
Replace Job ID below with your GNPS job ID:

In [1]:
! curl -d "" 'https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=fe25088a56a24d388306b1ef97b5e354&view=download_cytoscape_data' -o GNPS_output_graphML.zip
! unzip -d GNPS_output_graphML/ GNPS_output_graphML.zip

Download <i>in silico</i> structural annotation (Dereplicator, Varquest) data <br>
Replace Job IDs below with your Dereplicator or Varquest job IDs:

specify your NAP job ID:

In [1]:
nap_id1 = '94f8fc1dc4074a938a649ffbbc6d944e'
nap_id2 = 'f41eada3c6ec40f6b6a444a3b433acc1'

load libraries

In [2]:
from pyMolNetEnhancer import *
import pandas as pd
import os
import csv 
import json
from networkx import *
path = '/Applications/MarvinSuite/bin/' #path to MarvinSuite's molconvert used to convert SMILES to InchiKeys
os.environ['PATH'] += ':'+path

In [3]:
if 'clusterinfo_summary' in os.listdir('GNPS_output_graphML/') and 'DB_result' in os.listdir('GNPS_output_graphML/'):
    netfile = 'GNPS_output_graphML/clusterinfo_summary/' + str(os.listdir('GNPS_output_graphML/clusterinfo_summary/')[0]) 
    gnpslibfile = 'GNPS_output_graphML/DB_result/'+ str(os.listdir('GNPS_output_graphML/DB_result/')[0]) 
elif 'clusterinfosummarygroup_attributes_withIDs_withcomponentID' in os.listdir('GNPS_output_graphML/'):
    netfile = 'GNPS_output_graphML/clusterinfosummarygroup_attributes_withIDs_withcomponentID/' + str(os.listdir('GNPS_output_graphML/clusterinfosummarygroup_attributes_withIDs_withcomponentID/')[0])
    gnpslibfile = 'GNPS_output_graphML/result_specnets_DB/'+ str(os.listdir('GNPS_output_graphML/result_specnets_DB/')[0])
else:
    netfile = 'GNPS_output_graphML/clusterinfosummary/' + str(os.listdir('GNPS_output_graphML/clusterinfosummary/')[0])
    gnpslibfile = 'GNPS_output_graphML/result_specnets_DB/'+ str(os.listdir('GNPS_output_graphML/result_specnets_DB/')[0]) 

In [4]:
nap1 = pd.read_csv("http://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task="+nap_id1+"&block=main&file=final_out/node_attributes_table.tsv", sep = "\t")
nap2 = pd.read_csv("http://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task="+nap_id2+"&block=main&file=final_out/node_attributes_table.tsv", sep = "\t")
gnpslib = pd.read_csv(gnpslibfile, sep='\t')

add all chemical structural information output as dataframe items in list

In [5]:
matches = [gnpslib, nap1, nap2]

In [6]:
out = unique_smiles(matches)

In [7]:
out['df'].to_csv("SMILES.csv", quoting=csv.QUOTE_NONE, escapechar='&')

convert SMILES to InchiKeys

In [8]:
! molconvert inchikey:SAbs SMILES.csv{csv:strucSMILES} -o InchiKeys.txt

In [9]:
ikeys = pd.read_csv("InchiKeys.txt",  sep='\t',header = None)

out['df']["inchikey"] = ikeys
inchi_dic = make_inchidic(out)

ikeys.columns = ["InChIKey"]
ikeys.to_csv("InchiKeys.txt", quoting=csv.QUOTE_NONE, escapechar='&')

retrieve ClassyFire classifcations

In [10]:
get_classifications("InchiKeys.txt")

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


In [11]:
with open("all_json.json") as tweetfile:
    jsondic = json.loads(tweetfile.read())


df = make_classy_table(jsondic)
df = df.rename(columns = {'class':'CF_class','smiles':'SMILES'})
net = pd.read_csv(netfile,  sep='\t')


final = molfam_classes(net,df,inchi_dic)

In [12]:
final.head()

,cluster index,CF_componentindex,CF_NrNodes,CF_kingdom,CF_kingdom_score,CF_superclass,CF_superclass_score,CF_class,CF_class_score,CF_subclass,CF_subclass_score,CF_Dparent,CF_Dparent_score,CF_MFramework,CF_MFramework_score
0,344,5,5,Organic compounds,1,Organoheterocyclic compounds,0.269212,Carboxylic acids and derivatives,0.163784,NaN,0.181558,Coumarins and derivatives,0.061039,Aromatic heteropolycyclic compounds,0.52332
1,373,5,5,Organic compounds,1,Organoheterocyclic compounds,0.269212,Carboxylic acids and derivatives,0.163784,NaN,0.181558,Coumarins and derivatives,0.061039,Aromatic heteropolycyclic compounds,0.52332
2,617,5,5,Organic compounds,1,Organoheterocyclic compounds,0.269212,Carboxylic acids and derivatives,0.163784,NaN,0.181558,Coumarins and derivatives,0.061039,Aromatic heteropolycyclic compounds,0.52332
3,630,5,5,Organic compounds,1,Organoheterocyclic compounds,0.269212,Carboxylic acids and derivatives,0.163784,NaN,0.181558,Coumarins and derivatives,0.061039,Aromatic heteropolycyclic compounds,0.52332
4,260,5,5,Organic compounds,1,Organoheterocyclic compounds,0.269212,Carboxylic acids and derivatives,0.163784,NaN,0.181558,Coumarins and derivatives,0.061039,Aromatic heteropolycyclic compounds,0.52332


write ClassyFire results per molecular family to file. To visualize chemical classes within the mass spectral molecular network, this file can be imported as table into Cytoscape.

In [13]:
final.to_csv("ClassyFireResults_Network.txt", sep = '\t', index = False)

how many InChIKeys could not be classified? (percentage of total submitted InChIKeys)

In [14]:
len(set(list(ikeys.InChIKey)) - set(list(df.inchikey)))/len(set(list(ikeys.InChIKey)))

0.027922675667382633

which InChIKeys could not be classified?

In [15]:
set(list(ikeys.InChIKey)) - set(list(df.inchikey))

{'InChIKey=ABFNTRQPWNXUHA-UHFFFAOYSA-N',
 'InChIKey=AEKKQAOESHCKHZ-UHFFFAOYSA-N',
 'InChIKey=AFHWXJWAWBDGFQ-NWDGAFQWSA-O',
 'InChIKey=AGQKQQAJRFTOGL-KXQOOQHDSA-N',
 'InChIKey=AHVVIUYIKIQVDO-LDLOPFEMSA-N',
 'InChIKey=AHZZHULAOVWYNO-RRKDMDGFSA-N',
 'InChIKey=AIJDQMYBRDJHHT-ABAXQOPJSA-L',
 'InChIKey=AJLHXMZQAHWCRD-LEEZEASISA-O',
 'InChIKey=AJUBASUIRHJEOK-AQLSXGMYSA-Q',
 'InChIKey=AKEBJLCBLVMHQM-UHFFFAOYSA-O',
 'InChIKey=ALCRLVQDCOUCDC-HDICACEKSA-M',
 'InChIKey=AMBWIVCXWDCUBP-UHFFFAOYSA-N',
 'InChIKey=ANROOOJERCHKLE-XIFFEERXSA-N',
 'InChIKey=ASWLRRLLSBVSOH-SHUKQUCYSA-M',
 'InChIKey=BBUOWJVAOHPSOH-UWHLTILDSA-O',
 'InChIKey=BBYFWIHZNSBIGN-UHFFFAOYSA-O',
 'InChIKey=BCBQOXIPZHAELO-OVYGPGRDSA-O',
 'InChIKey=BEMVQPQCDJWLCN-FJRJUPPZSA-N',
 'InChIKey=BHVNHXGTBVOMHG-DGXTUMSLSA-O',
 'InChIKey=BHWKRHXUDXJMFO-XALMHBFYSA-N',
 'InChIKey=BMIBWFRRQPEUEB-MJGOQNOKSA-O',
 'InChIKey=BQQMPYJVCSLFGU-LFTNFPGXSA-N',
 'InChIKey=BSKKTUATLKRYJW-INIZCTEOSA-O',
 'InChIKey=BSPDZUGGMHYFFF-UHFFFAOYSA-N',
 'InChIKey=BSVYX

create graphML file

In [16]:
if any("FEATURE" in s for s in os.listdir('GNPS_output_graphML/')):
    graphMLfile = 'GNPS_output_graphML/' + [x for x in os.listdir('GNPS_output_graphML/') if 'FEATURE' in x][0]
    graphML = read_graphml(graphMLfile)
    graphML_classy = make_classyfire_graphml(graphML,final)
    nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)
elif any("METABOLOMICS" in s for s in os.listdir('GNPS_output_graphML/')):
    graphMLfile = 'GNPS_output_graphML/' + [x for x in os.listdir('GNPS_output_graphML/') if 'METABOLOMICS' in x][0]
    graphML = read_graphml(graphMLfile)
    graphML_classy = make_classyfire_graphml(graphML,final)
    nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)
else:
    print('There is no graphML file for this GNPS molecular network job')